In [3]:
import numpy as np
import pandas as pd
import os
import holidays

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split ,GridSearchCV,cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [4]:
planning_zone = 'AREA47'

# Define Canadian and Albertan public holidays
canadian_holidays = holidays.Canada(prov="AB")  # "AB" for Alberta

In [5]:
def clean_df(file_path: os.PathLike):
    df = pd.read_csv(file_path, encoding='unicode_escape')  # Read CSV file with proper encoding
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]  # Remove any "Unnamed" columns that might be auto-generated
    return df  # Return the cleaned DataFrame

In [6]:
# # Read CSVs and drop any unnamed index column
# # Create a new folder to save the merged CSV file
# os.makedirs('./Data/Merged', exist_ok=True)
# # Load cleaned CSVs (each file contains data for a specific time period)
# file_path = f"./Data/Area"
# all_weather_data = pd.concat([clean_df(f'{file_path}/{file}') for file in os.listdir(f'{file_path}')], ignore_index=True)
# # Save the merged dataframe as a new CSV file without index values
# all_weather_data.to_csv(f"./Data/Merged/area_weather_data.csv", index=False)  # `index=False` prevents writing index column to CSV
# print('Merged CSV file saved successfully!')
# print(f"Merged data has {all_weather_data.shape[0]} rows.")

In [7]:
def process_weather_station_data(df: pd.DataFrame, save: bool = False, columns = ['Station Name','Date', 'Air Temp. Inst. (Â°C)', 'Humidity Inst. (%)', 'Incoming Solar Rad. (W/m2)','Precip. (mm)', 'Wind Speed 10 m Syno. (km/h)', 'Wind Dir. 10 m Syno. (Â°)', 'Wind Speed 10 m Avg. (km/h)', 'Wind Dir. 10 m Avg. (Â°)',]) -> pd.DataFrame:
    """
    Get data for a specific asset.
    """
    try:
        df['Date'] = pd.to_datetime(df['Date (Local Standard Time)'], errors='coerce')        
        columns_to_drop = [column for column in df.columns if column not in columns]
        df.drop(columns=columns_to_drop, axis=1, inplace=True)
        print(df.isna().sum())
        df.fillna(method='ffill', inplace=True)
        if save:
            df.to_csv(f'./Data/Merged/area_weather_data', index=False)
            print('Processed CSV file saved successfully!')
        print("Miissing values filled")
        return df
    except Exception as e:
        print(f"No data found. Error: {e}")

In [8]:
def process_load_data(df: pd.DataFrame, save: bool = False, columns = ['Date', planning_zone]) -> pd.DataFrame:
    """
    Get data for a specific asset.
    """
    try:
        df.rename(columns={"DT_MST": "Date",}, inplace=True)
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
        columns_to_drop = [column for column in df.columns if column not in columns]
        df.drop(columns=columns_to_drop, axis=1, inplace=True)
        print(df.isna().sum())
        df.fillna(method='ffill', inplace=True)
        if save:
            df.to_csv(f'./Data/Merged/load_data_processed.csv', index=False)
            print('Processed CSV file saved successfully!')
        print("Miissing values filled")
        return df
    except Exception as e:
        print(f"No data found. Error: {e}")

In [9]:
load_data = process_load_data(clean_df('./Data/Merged/load_merged_data.csv'))
print(load_data.info())

Date      0
AREA47    0
dtype: int64
Miissing values filled
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122735 entries, 0 to 122734
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   Date    122735 non-null  datetime64[ns]
 1   AREA47  122735 non-null  float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 1.9 MB
None


C:\Users\nabil\AppData\Local\Temp\ipykernel_33728\907609487.py:11: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


In [10]:
# Read CSV file
weather_data = process_weather_station_data(clean_df(f'./Data/Merged/area_weather_data.csv'))

C:\Users\nabil\AppData\Local\Temp\ipykernel_33728\3648812536.py:2: DtypeWarning: Columns (4,7,10,11,19,22,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='unicode_escape')  # Read CSV file with proper encoding


Station Name                        0
Air Temp. Inst. (Â°C)               8
Humidity Inst. (%)                 19
Incoming Solar Rad. (W/m2)      21172
Precip. (mm)                       13
Wind Speed 10 m Syno. (km/h)       18
Wind Dir. 10 m Syno. (Â°)        1621
Wind Speed 10 m Avg. (km/h)        14
Wind Dir. 10 m Avg. (Â°)         1620
Date                                0
dtype: int64
Miissing values filled


C:\Users\nabil\AppData\Local\Temp\ipykernel_33728\3978271349.py:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


In [11]:
# Weather Data
# Rename columns to fix encoding issues
weather_data.rename(columns={
    "Air Temp. Inst. (Ã‚Â°C)": "Air Temp. Inst. (°C)",
    "Wind Dir. 10 m Syno. (Ã‚Â°)": "Wind Dir. 10 m Syno. (°)",
    "Wind Dir. 10 m Avg. (Ã‚Â°)": "Wind Dir. 10 m Avg. (°)"
}, inplace=True)

# Convert and fill missing values
weather_data["Incoming Solar Rad. (W/m2)"] = pd.to_numeric(weather_data["Incoming Solar Rad. (W/m2)"], errors="coerce")
weather_data["Incoming Solar Rad. (W/m2)"].fillna(weather_data["Incoming Solar Rad. (W/m2)"].median(), inplace=True)
weather_data["Date"] = pd.to_datetime(weather_data["Date"])


C:\Users\nabil\AppData\Local\Temp\ipykernel_33728\185971988.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  weather_data["Incoming Solar Rad. (W/m2)"].fillna(weather_data["Incoming Solar Rad. (W/m2)"].median(), inplace=True)


In [12]:
# merged_df = pd.merge(load_data, weather_data, on='Date', how='inner')
print(load_data.columns)
print(weather_data.columns)

Index(['Date', 'AREA47'], dtype='object')
Index(['Station Name', 'Air Temp. Inst. (Â°C)', 'Humidity Inst. (%)',
       'Incoming Solar Rad. (W/m2)', 'Precip. (mm)',
       'Wind Speed 10 m Syno. (km/h)', 'Wind Dir. 10 m Syno. (Â°)',
       'Wind Speed 10 m Avg. (km/h)', 'Wind Dir. 10 m Avg. (Â°)', 'Date'],
      dtype='object')


In [13]:
data = pd.merge(load_data, weather_data, on='Date', how='inner')

In [14]:
# Fix Datetime
data["Date"] = pd.to_datetime(data["Date"])  # Convert the date column to datetime
# data.set_index("Date", inplace=True)  # Set Date as index
data["Year"]  = data["Date"].dt.year
data["Month"]  = data["Date"].dt.month
data["Day"]  = data["Date"].dt.day
data["Hour"]  = data["Date"].dt.hour
data['is_weekend'] = data['Date'].dt.weekday >= 5
data['is_public_holiday'] = data['Date'].isin(canadian_holidays)
data.drop("Date", axis=1, inplace=True)  # Drop the original date column
data.drop("Station Name", axis=1, inplace=True)  # Drop the original date column
data.sort_index(inplace=True)  # Ensure the data is sorted by time

In [15]:
# Define seasons based on month
def get_season(month):
    if month in [12, 1, 2]:
        return "Winter"
    elif month in [3, 4, 5]:
        return "Spring"
    elif month in [6, 7, 8]:
        return "Summer"
    else:
        return "Fall"

data["Season"] = data["Month"].apply(get_season)
data = pd.get_dummies(data, columns=["Season"], drop_first=True)  # One-hot encode

In [16]:
# Cyclical encoding for month and hour
data["Month_sin"] = np.sin(2 * np.pi * data["Month"] / 12)
data["Month_cos"] = np.cos(2 * np.pi * data["Month"] / 12)
data["Hour_sin"] = np.sin(2 * np.pi * data["Hour"] / 24)
data["Hour_cos"] = np.cos(2 * np.pi * data["Hour"] / 24)

In [17]:
# Define the target variable: 'Volume' (the power generated)
data['Incoming Solar Rad. (W/m2)'] = pd.to_numeric(data['Incoming Solar Rad. (W/m2)'], errors='coerce')

# Handle 0 values in the target variable by applying log transformation with +1 (log(x+1)) to avoid issues with log(0)
y = data[planning_zone]  # Replace zero values with NaN for easier handling

# Create a binary indicator feature for zero values
data['Zero_Volume'] = (data[planning_zone] == 0).astype(int)

# Optionally, you can replace NaN values with the mean or median (if using log transformation and there are NaNs)
y = y.fillna(y.mean())  # You can also try using median depending on the data

# Apply log transformation to the target variable (log(x + 1))
y_log_transformed = np.log1p(y)

# Define the features: all columns except 'Volume' (and the new 'Zero_Volume' indicator)
X = data.drop([planning_zone], axis=1)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_log_transformed, test_size=0.2, random_state=42)

# Checking the data info to verify the changes
X.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315606 entries, 0 to 315605
Data columns (total 22 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Air Temp. Inst. (Â°C)         315606 non-null  float64
 1   Humidity Inst. (%)            315606 non-null  float64
 2   Incoming Solar Rad. (W/m2)    315606 non-null  float64
 3   Precip. (mm)                  315606 non-null  float64
 4   Wind Speed 10 m Syno. (km/h)  315606 non-null  float64
 5   Wind Dir. 10 m Syno. (Â°)     315606 non-null  float64
 6   Wind Speed 10 m Avg. (km/h)   315606 non-null  float64
 7   Wind Dir. 10 m Avg. (Â°)      315606 non-null  float64
 8   Year                          315606 non-null  int32  
 9   Month                         315606 non-null  int32  
 10  Day                           315606 non-null  int32  
 11  Hour                          315606 non-null  int32  
 12  is_weekend                    315606 non-nul

In [24]:
# Initialize the Random Forest model
# rf = RandomForestRegressor(random_state=42)
# Reduce complexity by limiting max depth and increasing min samples
rf = RandomForestRegressor(
    n_estimators=50,
    min_samples_leaf=10,  # Increase minimum samples at leaf
    max_depth=7,  # Reduce depth further
    oob_score=True,
    random_state=1
)
# Train the model on the training dataset (X_train, y_train)
rf.fit(X_train, y_train)

RandomForestRegressor(max_depth=7, min_samples_leaf=10, n_estimators=50,
                      oob_score=True, random_state=1)

In [25]:
from sklearn.metrics import mean_absolute_error, mean_squared_log_error, r2_score

# Predict the target values using the trained model on both training and test sets
y_train_pred = rf.predict(X_train)  # Predictions on the training set
y_test_pred = rf.predict(X_test)    # Predictions on the test set

# Calculate the Mean Absolute Error (MAE) for both training and test sets
# MAE measures the average absolute difference between predicted and actual values.
train_mae = mean_absolute_error(y_train, y_train_pred)
valid_mae = mean_absolute_error(y_test, y_test_pred)

# Calculate the Root Mean Squared Logarithmic Error (RMSLE) for both training and test sets
# RMSLE is often used for regression tasks when predictions can have a wide range.
# It penalizes underestimations more than overestimations.
train_rmsle = np.sqrt(mean_squared_log_error(y_train, y_train_pred))
valid_rmsle = np.sqrt(mean_squared_log_error(y_test, y_test_pred))

# Calculate the R-squared (R^2) score for both training and test sets
# R^2 measures how well the model explains the variance of the target variable.
# Higher values closer to 1 indicate better fit.
train_r2 = r2_score(y_train, y_train_pred)
valid_r2 = r2_score(y_test, y_test_pred)

# Create a dictionary to store and organize all the calculated results
results = {
    'Training MAE': train_mae,  # MAE for training set
    'Valid MAE': valid_mae,     # MAE for validation/test set
    'Training RMSLE': np.float64(train_rmsle),  # RMSLE for training set
    'Valid RMSLE': np.float64(valid_rmsle),    # RMSLE for validation/test set
    'Training R^2': train_r2,  # R^2 for training set
    'Valid R^2': valid_r2     # R^2 for validation/test set
}

# Print the results dictionary containing all the metrics
print(results)

{'Training MAE': 0.06388958404243703, 'Valid MAE': 0.06381248258350126, 'Training RMSLE': np.float64(0.016572590713025175), 'Valid RMSLE': np.float64(0.016559947543391514), 'Training R^2': 0.6667435659857253, 'Valid R^2': 0.6660539335753193}


In [26]:
# Get feature importances from the trained RandomForest model
feature_importances = rf.feature_importances_  # Importance of each feature in the model's prediction

# Get the names of the features (columns)
feature_names = X.columns  # Feature names from the training data

# Create a DataFrame to organize feature names and their corresponding importances for easy viewing
importance_df = pd.DataFrame({
    'Feature': feature_names,         # Feature names
    'Importance': feature_importances  # Corresponding importance values
})

# Sort the DataFrame by the 'Importance' column in descending order to see the most important features first
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Print the DataFrame containing feature names and their importance values
print(importance_df)

                         Feature    Importance
8                           Year  3.491372e-01
2     Incoming Solar Rad. (W/m2)  2.492609e-01
0          Air Temp. Inst. (Â°C)  1.214849e-01
15                 Season_Summer  8.583236e-02
16                 Season_Winter  4.571070e-02
9                          Month  3.710014e-02
12                    is_weekend  3.077796e-02
19                      Hour_sin  2.228697e-02
11                          Hour  2.174693e-02
20                      Hour_cos  1.740387e-02
18                     Month_cos  7.113252e-03
17                     Month_sin  6.947026e-03
10                           Day  3.368435e-03
6    Wind Speed 10 m Avg. (km/h)  1.195332e-03
4   Wind Speed 10 m Syno. (km/h)  3.574444e-04
1             Humidity Inst. (%)  1.664295e-04
14                 Season_Spring  6.600006e-05
5      Wind Dir. 10 m Syno. (Â°)  2.373956e-05
7       Wind Dir. 10 m Avg. (Â°)  1.947313e-05
3                   Precip. (mm)  9.447331e-07
13           

In [27]:
rf.predict(X_test)

array([4.26664659, 4.104227  , 4.26664659, ..., 4.37479781, 4.140425  ,
       4.33847219])

In [28]:
y_test

261397    4.157496
89361     4.263641
210139    4.190620
168379    4.242745
56095     4.303373
            ...   
4385      4.416559
139746    4.194551
54312     4.362709
285733    4.179044
33024     4.232089
Name: AREA47, Length: 63122, dtype: float64

In [29]:
from sklearn.model_selection import cross_val_score
import numpy as np


# Perform 5-fold cross-validation
cv_scores = cross_val_score(rf, X, y, cv=5, scoring='neg_mean_absolute_error')

# Convert negative MAE to positive
cv_scores = -cv_scores

# Print out the cross-validation scores
print("Cross-validation MAE scores:", cv_scores)
print("Average Cross-validation MAE:", np.mean(cv_scores))


Cross-validation MAE scores: [9.0695274  7.30333774 6.04724243 5.4985952  5.24335068]
Average Cross-validation MAE: 6.632410691612162


In [30]:
y_min = y.min()
y_max = y.max()

# Print the range
print(f"Target variable range: {y_min} to {y_max}")

Target variable range: 32.5365603 to 106.587303
